## Домашнее задание к лекции «Основы парсинга и работы с API»

### Задание 1.
**Обязательная часть**  
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = [‘python’, ‘парсинг’]

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [1]:
import requests

In [2]:
import time

In [3]:
from bs4 import BeautifulSoup

In [4]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text)

In [5]:
soup

<!DOCTYPE html>
<html data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D" lang="ru">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,viewport-fit=cover" name="viewport"/>
<title>Все публикации подряд / Хабр</title>
<style>
    /* cyrillic-ext */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveSxf6TF0.woff2) format('woff2');
      unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;
    }

    /* cyrillic */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveQhf6TF0.woff2) format('woff2');
      unicode-range: U+0400-045F, U+0490-0491, U+04B0-04B1, U+2116;
    }

    /* latin-ext */
    @font-face {
      f

In [6]:
posts = soup.find_all('article', class_='tm-articles-list__item')

In [7]:
posts

[<article class="tm-articles-list__item" data-navigatable="" id="573618" tabindex="0"><div class="tm-article-snippet"><div class="tm-article-snippet__meta-container"><div class="tm-article-snippet__meta"><span class="tm-user-info tm-article-snippet__author"><a class="tm-user-info__userpic" href="/ru/users/third112/" title="third112"><div class="tm-entity-image"><img alt="" class="tm-entity-image__pic" height="24" loading="lazy" src="//habrastorage.org/r/w32/getpro/habr/avatars/4ea/3d6/cba/4ea3d6cbacc7a7b38328ad88cd4110bb.jpg" width="24"/></div></a><span class="tm-user-info__user"><a class="tm-user-info__username" href="/ru/users/third112/">
       third112
     </a></span></span><span class="tm-article-snippet__datetime-published"><time datetime="2021-08-18T19:18:23.000Z" title="2021-08-18, 22:18">сегодня в 22:18</time></span></div><!-- --></div><h2 class="tm-article-snippet__title tm-article-snippet__title_h2"><a class="tm-article-snippet__title-link" data-article-link="" href="/ru/po

In [8]:
len(posts)

20

In [10]:
for post in posts:
    hubs = post.find_all('div', class_='article-formatted-body article-formatted-body_version-1') + post.find_all('div', class_='article-formatted-body article-formatted-body_version-2')
    print(len(hubs))
    #проверяем что извлекли из каждой новости её тело, т.е. хабы не пустые. классов оказалось два!

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [12]:
hub = posts[0].find_all('div', class_='article-formatted-body article-formatted-body_version-1')
hub

[<div class="article-formatted-body article-formatted-body_version-1"><img src="https://habrastorage.org/webt/72/4b/ac/724bacaqwdn9knndmxwdju8n-1i.jpeg"/><br/>
 <br/>
 Не один десяток лет программирую. И временами наступаю на одни грабли: закрывающие скобки. Всегда нужно соответствие открывающим. Будь то выражение или программные скобки в функции. IDE обязательно укажет на ошибку, но раздражает!<br/></div>]

In [13]:
hub_lower = hub[0].text.lower()
hub_lower

'\n\r\nне один десяток лет программирую. и временами наступаю на одни грабли: закрывающие скобки. всегда нужно соответствие открывающим. будь то выражение или программные скобки в функции. ide обязательно укажет на ошибку, но раздражает!'

In [ ]:
# для примера

In [15]:
words = ['fgg', 'kd', 'ide']

In [16]:
any([word in hub_lower for word in words])

True

In [17]:
import pandas as pd

In [18]:
news = pd.DataFrame()
if any([word in hub_lower for word in words]):
    link = 'https://habr.com' + posts[0].find('a', class_='tm-article-snippet__readmore').attrs.get('href')
    title = posts[0].find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').text
    date = pd.to_datetime(posts[0].find('time').attrs.get('title')).date()
#     print(title)
#     print(date)
#     print(link)
row = {'date': date, 'title': title, 'link': link}
news = pd.concat([news, pd.DataFrame([row])])  
news    

,date,title,link
0,2021-08-18,Мелкая (?) проблема закрывающих скобок в ЯП,https://habr.com/ru/post/573618/


In [ ]:
# достаем ссылки

In [19]:
a = posts[0].find('a', class_='tm-article-snippet__readmore').attrs.get('href')
a

'/ru/post/573618/'

In [21]:
link = 'https://habr.com/' + a
link

'https://habr.com//ru/post/573618/'

In [22]:
# достаем заголовок

In [23]:
title = posts[0].find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2')
title.text

'Мелкая (?) проблема закрывающих скобок в ЯП'

In [ ]:
# достаем дату

In [24]:
date = posts[0].find('time').attrs.get('title')
date

'2021-08-18, 22:18'

In [26]:
date = pd.to_datetime(date).date()
type(date)
print(date)

2021-08-18


In [25]:
# ИТОГ!

In [27]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text)

In [28]:
posts = soup.find_all('article', class_='tm-articles-list__item')

In [29]:
posts

[<article class="tm-articles-list__item" data-navigatable="" id="573618" tabindex="0"><div class="tm-article-snippet"><div class="tm-article-snippet__meta-container"><div class="tm-article-snippet__meta"><span class="tm-user-info tm-article-snippet__author"><a class="tm-user-info__userpic" href="/ru/users/third112/" title="third112"><div class="tm-entity-image"><img alt="" class="tm-entity-image__pic" height="24" loading="lazy" src="//habrastorage.org/r/w32/getpro/habr/avatars/4ea/3d6/cba/4ea3d6cbacc7a7b38328ad88cd4110bb.jpg" width="24"/></div></a><span class="tm-user-info__user"><a class="tm-user-info__username" href="/ru/users/third112/">
       third112
     </a></span></span><span class="tm-article-snippet__datetime-published"><time datetime="2021-08-18T19:18:23.000Z" title="2021-08-18, 22:18">сегодня в 22:18</time></span></div><!-- --></div><h2 class="tm-article-snippet__title tm-article-snippet__title_h2"><a class="tm-article-snippet__title-link" data-article-link="" href="/ru/po

In [32]:
keywords = ['python', 'парсинг']
# в сегодняшней ленте не нашлось, добавим слово
keywords = ['python', 'парсинг', 'токен']


In [33]:
news = pd.DataFrame()
for post in posts:
    hubs = post.find_all('div', class_='article-formatted-body article-formatted-body_version-1') \
    + post.find_all('div', class_='article-formatted-body article-formatted-body_version-2')
    for hub in hubs:
            hub_lower = hub.text.lower()
            if any([word in hub_lower for word in keywords]):
                link = 'https://habr.com' + post.find('a', class_='tm-article-snippet__readmore').attrs.get('href')
                title = post.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').text
                date = pd.to_datetime(post.find('time').attrs.get('title')).date()
                row = {'date': date, 'title': title, 'link': link}
                news = pd.concat([news, pd.DataFrame([row])])  
news

,date,title,link
0,2021-08-18,Токенизация сообществ,https://habr.com/ru/post/573536/


**Дополнительная часть (необязательная)**  
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [ ]:
# получим текст новости

In [34]:
for post in posts:
    link = 'https://habr.com' + post.find('a', class_='tm-article-snippet__readmore').attrs.get('href')
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.3)
    text = soup.find('div', class_='tm-article-body').text
#     text = soup.find('div', class_='article-formatted-body article-formatted-body_version-2').text
    print(text)
    break



Не один десяток лет программирую. И временами наступаю на одни грабли: закрывающие скобки. Всегда нужно соответствие открывающим. Будь то выражение или программные скобки в функции. IDE обязательно укажет на ошибку, но раздражает!

 В Pascal 8000 для ЕС ЭВМ была конструкция FORALL – чем делать тупой перебор множества S:

for i :=1 to 250 do
 if i in S then
 … {do something}

можно было написать:

forall i in S do

… {something}

что выглядит более эффективно. 

В книге Дал У., Дейкстра Э., Хоор К., Структурное программирование. М.:«Мир», 1975,
для множеств была предложена функция next, возвращающая следующий член множества. Не понятно почему Вирт не реализовал ее в своем Паскале, и далее ее обошли вниманием в других компиляторах Паскаля. Реализация не сложная, но это будет уход от темы.

Возвращаясь к закрывающим скобкам – ИМХО в ЯП добавлять слово ALL. Нпр., “)all” пусть закроет все скобки в выражении, endall закроет все программные скобки в паскалевской процедуре, “}all” — закроет 

In [57]:
link = 'https://habr.com' + posts[0].find('a', class_='tm-article-snippet__readmore').attrs.get('href')
soup = BeautifulSoup(requests.get(link).text)
text = soup.find('div', class_='tm-article-body').text
text

'\n\r\nНе один десяток лет программирую. И временами наступаю на одни грабли: закрывающие скобки. Всегда нужно соответствие открывающим. Будь то выражение или программные скобки в функции. IDE обязательно укажет на ошибку, но раздражает!\n\r\n В Pascal 8000 для ЕС ЭВМ была конструкция FORALL – чем делать тупой перебор множества S:\n\nfor i :=1 to 250 do\n if i in S then\n … {do something}\n\r\nможно было написать:\n\nforall i in S do\n\n… {something}\n\r\nчто выглядит более эффективно. \n\r\nВ книге Дал У., Дейкстра Э., Хоор К., Структурное программирование. М.:«Мир», 1975,\r\nдля множеств была предложена функция next, возвращающая следующий член множества. Не понятно почему Вирт не реализовал ее в своем Паскале, и далее ее обошли вниманием в других компиляторах Паскаля. Реализация не сложная, но это будет уход от темы.\n\r\nВозвращаясь к закрывающим скобкам – ИМХО в ЯП добавлять слово ALL. Нпр., “)all” пусть закроет все скобки в выражении, endall закроет все программные скобки в паска

In [62]:
full_news = pd.DataFrame()
for post in posts:
    link = 'https://habr.com' + post.find('a', class_='tm-article-snippet__readmore').attrs.get('href')
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.3)
    text = soup.find('div', class_='tm-article-body').text
#     text = soup.find('div', class_='article-formatted-body article-formatted-body_version-1') \
#     + soup.find('div', class_='article-formatted-body article-formatted-body_version-2')
    if any([word in text.lower() for word in keywords]):
        title = soup.find('h1', class_='tm-article-snippet__title tm-article-snippet__title_h1').text
        date = pd.to_datetime(soup.find('time').attrs.get('title')).date()
        row = {'date': date, 'title': title, 'link': link, 'text':text}
        full_news = pd.concat([full_news, pd.DataFrame([row])])
    
full_news

,date,title,link,text
0,2021-08-18,Первый Компьютерный ЕГЭ 2021,https://habr.com/ru/post/573580/,Всем привет! В этом году я закончил 11 класс и...
0,2021-08-18,5 инструментов для удаленной команды Automatio...,https://habr.com/ru/post/573540/,"Кому-то этот пост покажется запоздавшим, кому-..."
0,2021-08-18,Токенизация сообществ,https://habr.com/ru/post/573536/,Токенизировать можно не только коммерческие ко...


In [ ]:
# при поиске по полному тексту нашли ещё две новости с ключевыми словами

### Задание 2.
**Обязательная часть**  
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

**Подсказка**: сервис работает при помощи “скрытого” API. Внимательно изучите post-запросы.

In [4]:
import json

In [5]:
headers = {
# 'Accept': 'application/json, text/plain, */*',
# 'Accept-Encoding': 'gzip, deflate, br',
# 'Accept-Language': 'ru,en;q=0.9',
# 'Connection': 'keep-alive',
# 'Content-Length': '33',
# 'Content-Type': 'application/json;charset=UTF-8',
# 'Host': 'identityprotection.avast.com',
# 'Origin': 'https://www.avast.com',
# 'Referer': 'https://www.avast.com/',
# 'sec-ch-ua': '" Not;A Brand";v="99", "Yandex";v="91", "Chromium";v="91"',
# 'sec-ch-ua-mobile': '?0',
# 'Sec-Fetch-Dest': 'empty',
# 'Sec-Fetch-Mode': 'cors',
# 'Sec-Fetch-Site': 'same-site',
# 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 YaBrowser/21.6.4.787 Yowser/2.5 Safari/537.36',
'Vaar-Header-App-Build-Version': '1.0.0',
'Vaar-Header-App-Product': 'hackcheck-web-avast',
'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
'Vaar-Version': '0'}
headers

{'Vaar-Header-App-Build-Version': '1.0.0',
 'Vaar-Header-App-Product': 'hackcheck-web-avast',
 'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
 'Vaar-Version': '0'}

In [7]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
url

'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [59]:
data_json = {"emailAddresses": ["123456@ya.com", "qwerty@ya.com"]}
type(data_json)

dict

In [60]:
request = requests.post(url, json = data_json, headers = headers)
request

<Response [200]>

In [61]:
request.json()

{'breaches': {'3': {'breachId': 3,
   'site': 'adobe.com',
   'recordsCount': 152046506,
   'description': "In October of 2013, criminals penetrated Adobe's corporate network and the stole source code for several of its software products. The affected products included Adobe's ColdFusion web application platform as well as the Acrobat suite of products. Adobe asserts that criminals also accessed nearly three-million customer credit card records and stole login data for an undisclosed number of Adobe user accounts.",
   'publishDate': '2016-10-21T00:00:00Z',
   'statistics': {'usernames': 0,
    'passwords': 129430596,
    'emails': 152046506}},
  '15': {'breachId': 15,
   'site': 'imesh.com',
   'recordsCount': 50913222,
   'description': 'In June 2016, a cache of over 51 million user credentials from the online sharing service iMesh appeared for sale on a dark web marketplace.  The database contained user email addresses, usernames, passwords, IP addresses and location.',
   'publishD

In [62]:
request.json().keys()

dict_keys(['breaches', 'data', 'summary'])

In [63]:
request.json()['breaches']['3'].keys()

dict_keys(['breachId', 'site', 'recordsCount', 'description', 'publishDate', 'statistics'])

In [64]:
request.json()['breaches']['3']['publishDate'][:10]

'2016-10-21'

In [127]:
import pandas as pd
df = pd.DataFrame(request.json()['breaches']).T.reset_index()[['breachId', 'publishDate', 'site', 'description']]

In [128]:
df.publishDate =  df.publishDate[0][:10]

In [129]:
df # итоговый датафрейм

,breachId,publishDate,site,description
0,3,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
1,15,2016-10-21,imesh.com,"In June 2016, a cache of over 51 million user ..."
2,60,2016-10-21,qip.ru,"In 2011, Russian instant messaging service pro..."
3,88,2016-10-21,ir.netease.com,"In October 2015, the Chinese internet and gami..."
4,3772,2016-10-21,evony.com,"In July 2016, gaming site Evony's user databas..."
5,3775,2016-10-21,51cto.com,"In December 2013, 51CTO's user database was al..."
6,15996,2016-10-21,vng.com.vn,"In 2018, VNG Corporation's database was allege..."
7,16768,2016-10-21,canva.com,"In May 2019, graphic-design site Canva's datab..."
8,17009,2016-10-21,zynga.com,"In September 2019, the game developer Zynga wa..."
9,17125,2016-10-21,gigasize.com,"At an unconfirmed date, the file sharing site ..."


In [ ]:
# какие уточки к какой почте относятся

In [125]:
df_total_breaches = pd.DataFrame(request.json()['summary']).T
df_total_breaches

,breaches
123456@ya.com,"[3, 15, 60, 88, 3772, 3775, 15996, 17009]"
qwerty@ya.com,"[15, 16768, 17009, 17125, 37372]"


In [ ]:
# фильтр по конкретной почте

In [130]:
df[df['breachId'].isin(df_total_breaches['breaches']['123456@ya.com'])]

,breachId,publishDate,site,description
0,3,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
1,15,2016-10-21,imesh.com,"In June 2016, a cache of over 51 million user ..."
2,60,2016-10-21,qip.ru,"In 2011, Russian instant messaging service pro..."
3,88,2016-10-21,ir.netease.com,"In October 2015, the Chinese internet and gami..."
4,3772,2016-10-21,evony.com,"In July 2016, gaming site Evony's user databas..."
5,3775,2016-10-21,51cto.com,"In December 2013, 51CTO's user database was al..."
6,15996,2016-10-21,vng.com.vn,"In 2018, VNG Corporation's database was allege..."
8,17009,2016-10-21,zynga.com,"In September 2019, the game developer Zynga wa..."
